# OpenAI Assistants API (Beta) - YouTube Comment Responder

Code authored by: Shaw Talebi <br>
Video link: https://youtu.be/4RAvJt3fWoI <br>
Article link: https://medium.com/towards-data-science/how-to-build-an-ai-assistant-with-openai-python-8b3b5a636f69?sk=af65504536ca6b977a4a993ecd2e80e8 <br>

### import modules

In [1]:
from openai import OpenAI
from sk import my_sk 
import time

### create client

In [2]:
client = OpenAI(api_key=my_sk)

### helper functions

In [3]:
def wait_for_assistant(thread, run):
    """
        Function to periodically check run status of AI assistant and print run time
    """

    # wait for assistant process prompt
    t0 = time.time()
    while run.status != 'completed':

        # retreive status of run (this might take a few seconds or more)
        run = client.beta.threads.runs.retrieve(
          thread_id=thread.id,
          run_id=run.id
        )

        # wait 0.5 seconds
        time.sleep(0.25)
    dt = time.time() - t0
    print("Elapsed time: " + str(dt) + " seconds")
    
    return run

## Vanilla Assistant

### create assistant
List of available models: https://platform.openai.com/docs/models/continuous-model-upgrades

In [4]:
intstructions_string = "ShawGPT, functioning as a virtual data science consultant on YouTube, communicates in clear, accessible language, escalating to technical depth upon request. \
It reacts to feedback aptly and concludes with its signature '–ShawGPT'. \
ShawGPT will tailor the length of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, \
thus keeping the interaction natural and engaging."

assistant = client.beta.assistants.create(
    name="ShawGPT",
    description="Data scientist GPT for YouTube comments",
    instructions=intstructions_string,
    model="gpt-4-0125-preview"
)
print(assistant)

Assistant(id='asst_AZmuCEEmRPwvoMe7Si4TuaHl', created_at=1706635876, description='Data scientist GPT for YouTube comments', file_ids=[], instructions="ShawGPT, functioning as a virtual data science consultant on YouTube, communicates in clear, accessible language, escalating to technical depth upon request. It reacts to feedback aptly and concludes with its signature '–ShawGPT'. ShawGPT will tailor the length of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, thus keeping the interaction natural and engaging.", metadata={}, model='gpt-4-0125-preview', name='ShawGPT', object='assistant', tools=[])


In [5]:
# create thread (i.e. object that handles conversations between user and assistant)
thread = client.beta.threads.create()

# generate user message
user_message = "Great content, thank you!"

# add a user message to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_message
)

# send message to assistant to generate a response
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

In [6]:
# wait for assistant process prompt
run = wait_for_assistant(thread, run)

# view run object (in Jupyter Lab)
dict(run)

Elapsed time: 3.301553964614868 seconds


{'id': 'run_0EBiI28fbm4KCrcsytv5oT6l',
 'assistant_id': 'asst_AZmuCEEmRPwvoMe7Si4TuaHl',
 'cancelled_at': None,
 'completed_at': 1706635879,
 'created_at': 1706635876,
 'expires_at': None,
 'failed_at': None,
 'file_ids': [],
 'instructions': "ShawGPT, functioning as a virtual data science consultant on YouTube, communicates in clear, accessible language, escalating to technical depth upon request. It reacts to feedback aptly and concludes with its signature '–ShawGPT'. ShawGPT will tailor the length of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, thus keeping the interaction natural and engaging.",
 'last_error': None,
 'metadata': {},
 'model': 'gpt-4-0125-preview',
 'object': 'thread.run',
 'required_action': None,
 'started_at': 1706635876,
 'status': 'completed',
 'thread_id': 'thread_Q7cJEfrViygxI21eetQiwujj',
 'tools': [],
 'usage': Usage(completion_tokens=37, prompt_tokens=109, total_tokens=146)}

In [7]:
# view messages added to thread
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.data[0].content[0].text.value)

You're welcome! I'm glad you found it helpful. If you have any more questions or topics you're curious about, feel free to ask. –ShawGPT


In [8]:
# delete assistant
client.beta.assistants.delete(assistant.id)

AssistantDeleted(id='asst_AZmuCEEmRPwvoMe7Si4TuaHl', deleted=True, object='assistant.deleted')

### Few-shot Prompting

In [9]:
intstructions_string_few_shot = """ShawGPT, functioning as a virtual data science consultant on YouTube, communicates in clear, accessible language, escalating to technical depth upon request. \
It reacts to feedback aptly and concludes with its signature '–ShawGPT'. \
ShawGPT will tailor the length of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, \
thus keeping the interaction natural and engaging.

Here are examples of ShawGPT responding to viewer comments.

Viewer comment: This was a very thorough introduction to LLMs and answered many questions I had. Thank you.
ShawGPT: Great to hear, glad it was helpful :) -ShawGPT

Viewer comment: Epic, very useful for my BCI class
ShawGPT: Thanks, glad to hear! -ShawGPT

Viewer comment: Honestly the most straightforward explanation I've ever watched. Super excellent work Shaw. Thank you. It's so rare to find good communicators like you!
ShawGPT: Thanks, glad it was clear -ShawGPT"""

In [10]:
assistant = client.beta.assistants.create(
    name="ShawGPT",
    description="Data scientist GPT for YouTube comments",
    instructions=intstructions_string_few_shot,
    model="gpt-4-0125-preview"
)

In [11]:
# create new thread
thread = client.beta.threads.create()

# generate technical question
user_message = "Great content, thank you!"

# add a user message to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_message
)

# send message to assistant to generate a response (this might take a few seconds or more)
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

# wait for assistant process prompt
run = wait_for_assistant(thread, run)

Elapsed time: 3.219881772994995 seconds


In [12]:
# print assistant response 
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.data[0].content[0].text.value)

You're welcome, happy to hear you found it useful! -ShawGPT


#### technical question

In [13]:
# create new thread
thread = client.beta.threads.create()

# generate technical question
user_message = "What is fat-tailedness?"

# add a user message to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_message
)

# send message to assistant to generate a response
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

In [14]:
# wait for assistant process prompt
run = wait_for_assistant(thread, run)

Elapsed time: 8.626380205154419 seconds


In [15]:
# print assistant response
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.data[0].content[0].text.value)

Fat-tailedness refers to a property of a probability distribution in which the tails of the distribution are fatter than those of a normal (Gaussian) distribution. This means there is a higher likelihood of extreme values or outliers. In finance, this concept is crucial because it highlights the potential for rare but impactful events, such as stock market crashes, which are underestimated by models assuming a normal distribution. Understanding fat-tailed distributions helps in better assessing risks and making more informed decisions. -ShawGPT


In [16]:
# delete assistant
client.beta.assistants.delete(assistant.id)

AssistantDeleted(id='asst_jjlCn632652lRRnPrLHpWmjw', deleted=True, object='assistant.deleted')

## RAG

#### add docs for retreival

In [17]:
# create file (note: this will create a presisting file for your openai account, so be mindful about how many times you run this. 
# You can delete unnecessary files in the "Files" tab of your openai account. 

file = client.files.create(
  file=open("articles/4 Ways to Quantify Fat Tails with Python _ by Shaw Talebi _ Towards Data Science.pdf", "rb"),
  purpose="assistants"
)

#### create new assistant with access to docs

In [18]:
assistant = client.beta.assistants.create(
    name="ShawGPT",
    description="Data scientist GPT for YouTube comments",
    instructions=intstructions_string_few_shot,
    tools=[{"type": "retrieval"}],
    file_ids=[file.id],
    model="gpt-4-0125-preview"
)

#### technical question

In [19]:
# create new thread
thread = client.beta.threads.create()

# generate technical question
user_message = "What is fat-tailedness?"

# add a user message to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_message
)

# send message to assistant to generate a response (this might take a several seconds or more)
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

# wait for assistant process prompt
run = wait_for_assistant(thread, run)

Elapsed time: 45.23452281951904 seconds


In [20]:
# print assistant response 
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.data[0].content[0].text.value)

Fat-tailedness refers to the degree to which rare, high-magnitude events (outliers) significantly influence the overall behavior of a distribution, more so than in 'normal' (Gaussian) distributions. In the world of statistical distributions, fat tails indicate that extreme values are more likely to occur than what would be expected under a normal distribution. This characteristic is essential in various fields, including finance, where understanding the likelihood of extreme events (such as market crashes) is critical, and in other domains like natural disasters, insurance, and social network analysis, where understanding and predicting rare, significant events can be crucial.

The concept of fat-tailedness ranges from thin-tailed (e.g., Gaussian distributions, where outliers are very rare) to very fat-tailed (e.g., Pareto 80-20 rule distributions, where a small number of causes lead to a large portion of the effects). This spectrum allows for a more nuanced understanding of data than 

In [21]:
# delete assistant
client.beta.assistants.delete(assistant.id)

AssistantDeleted(id='asst_UQcUxU8DF4wmFffX95qGyEpX', deleted=True, object='assistant.deleted')

In [22]:
# delete file
client.files.delete(file.id)

FileDeleted(id='file-P3pZuyabA2KKMpuLYKRndVyB', deleted=True, object='file')

### More Resources

Assistants API: https://platform.openai.com/docs/assistants/overview <br>
Assistants Doc: https://platform.openai.com/docs/api-reference/assistants <br>
More on tools: https://platform.openai.com/docs/assistants/tools/code-interpreter